In [ ]:
import os
import numpy as np
import re
from statannotations.Annotator import Annotator
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [ ]:
assert int(pd.__version__[0]) < 2, 'Please < 2 required for statannotations'

In [ ]:
data_dir = '/Users/jk1/temp/cereblink/data_saving/exclude_nan_outcome_False'
pvals_dir = '/Users/jk1/temp/cereblink/data_saving/exclude_nan_outcome_False'
output_dir = '/Users/jk1/Downloads'

# Load pupillometry data

In [ ]:
data_filenames = [f for f in os.listdir(data_dir) if f.endswith('.csv') and 'timebin' in f and 'reassembled_pupillometry' in f]

pupillometry_df = pd.DataFrame()
for data_filename in data_filenames:
    # find timebin size with regex identifying pattern : _xh_
    timebin_size = int(re.search(r'_(\d+)h_', data_filename).group(1))
    data_is_normalized = int(('normalized' in data_filename) or ('normalised' in data_filename))
    outcome = '_'.join(data_filename.split('_')[0:2])

    df = pd.read_csv(os.path.join(data_dir, data_filename))
    df['timebin_size'] = timebin_size
    df['normalized'] = data_is_normalized
    df['outcome'] = outcome
    pupillometry_df = pd.concat([pupillometry_df, df], axis=0)
    
pupillometry_df = pupillometry_df.reset_index(drop=True)

In [ ]:
pupillometry_df.head()

# Load p-values

In [ ]:
pvals_filenames = [f for f in os.listdir(pvals_dir) if f.endswith('.csv') and 'pvals' in f]

pvals_df = pd.DataFrame()
for pvals_filename in pvals_filenames:
    # find timebin size with regex identifying pattern : _xh_
    timebin_size = int(re.search(r'_(\d+)h_', pvals_filename).group(1))
    data_is_normalized = int(('normalized' in pvals_filename) or ('normalised' in pvals_filename))
    using_span = int(('with_span' in pvals_filename))
    outcome = '_'.join(pvals_filename.split('_')[0:2])

    df = pd.read_csv(os.path.join(pvals_dir, pvals_filename), index_col=0)
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'metric'}, inplace=True)
    df['timebin_size'] = timebin_size
    df['normalized'] = data_is_normalized
    df['using_span'] = using_span
    df['outcome'] = outcome
    pvals_df = pd.concat([pvals_df, df], axis=0)
    
pvals_df = pvals_df[pvals_df['using_span'] == 0]

# Plot pupillometry data

In [ ]:
all_colors_palette = sns.color_palette(['#f61067', '#049b9a', '#012D98', '#a76dfe', '#FFA987', '#E2FDFF'], n_colors=6)
all_colors_palette

In [ ]:
selected_colors_palette = sns.color_palette(['#FFA987', '#049b9a'], n_colors=2)
selected_colors_palette

In [ ]:
pupillometry_metrics = ['NPI', 'CV']
inter_eye_metrics = ['mean', 'min', 'max', 'delta']
# combine to get all metrics
single_timepoint_metrics = [f'{metric}_inter_eye_{metric_type}' for metric in pupillometry_metrics for metric_type in
                            inter_eye_metrics]
over_time_metrics = ['max', 'min', 'median']
# combine to get all metrics
timebin_metrics = [f'{metric}_timebin_{metric_type}' for metric in single_timepoint_metrics for metric_type in
                   over_time_metrics]

In [ ]:
def plot_metric_distributions_over_timebins(df, over_time_metrics, timebin_metrics, plot_type='box', pvals=None, pval_method='adjusted_pval', alpha=0.5, 
                                            plot_legend = True, tick_label_size = 11,
                                            label_font_size = 13, fig=None):
    n_columns = len(over_time_metrics)
    n_rows = int(np.ceil(len(timebin_metrics) / n_columns))
        
    if fig is None:
        fig, axes = plt.subplots(n_rows, n_columns, figsize=(n_columns * 20/3, n_rows * 60/8))
    else:
        axes = fig.subplots(n_rows, n_columns)
    # ensure axes is a 2D array
    if n_rows == 1:
        axes = axes[np.newaxis, :]
    custom_palette = {0: '#FFA987', 1: '#049b9a'}
    
    for i, metric in enumerate(timebin_metrics):
        plot_params = {
                'data': df,
                'x': 'timebin_size',
                'y': metric,
                'hue': 'label',
                'palette': custom_palette
            }
        if plot_type == 'violin':
            plot_params['split'] = True
            plot_params['gap'] = 0.1
            sns.violinplot(**plot_params, ax=axes[i // n_columns, i % n_columns])
        elif plot_type == 'box':
            plot_params['showfliers'] = False
            sns.boxplot(**plot_params, ax=axes[i // n_columns, i % n_columns])
        else:
            print('plot type not recognized')
        axes[i // n_columns, i % n_columns].set_title(metric)
        axes[i // n_columns, i % n_columns].set_ylabel('')
        axes[i // n_columns, i % n_columns].set_xlabel('Timebin size (hours)', fontsize=label_font_size)
        axes[i // n_columns, i % n_columns].tick_params('x', labelsize=tick_label_size)
        axes[i // n_columns, i % n_columns].tick_params('y', labelsize=tick_label_size)

        # added transparency to the boxplot
        for patch in axes[i // n_columns, i % n_columns].patches:
            r, g, b, a = patch.get_facecolor()
            patch.set_facecolor((r, g, b, alpha))
        
        if pvals is not None:
            pvals_metric = pvals[pvals['metric'] == metric]
            pvals_metric = pvals_metric.sort_values(by='timebin_size')

            timebin_values = pvals_metric['timebin_size'].unique()
            # use statannotations to display p-values
            pairs = (
                [(timebin_values[0], 0), (timebin_values[0], 1)],
                [(timebin_values[1], 0), (timebin_values[1], 1)],
                [(timebin_values[2], 0), (timebin_values[2], 1)],
                [(timebin_values[3], 0), (timebin_values[3], 1)],
            )
            
            # Add annotations
            annotator = Annotator(axes[i // n_columns, i % n_columns], pairs, **plot_params, verbose=False)
            annotator.set_pvalues(pvals_metric[pval_method].values)
            annotator.annotate()
            
        if plot_legend:
            handles, _ = axes[i // n_columns, i % n_columns].get_legend_handles_labels()
            # set alpha in handles
            for handle in handles:
                handle.set_alpha(alpha)
            labels = ['No DCI', 'DCI']
            axes[i // n_columns, i % n_columns].legend(handles, labels, title='', loc='lower right', facecolor='white', framealpha=0.9,
                                                       fontsize=tick_label_size, title_fontsize=label_font_size)
            
    return fig, axes

DCI ischemia

In [ ]:
target = 'DCI_ischemia'
fig1, axes = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 0) 
                                                    & (pupillometry_df['outcome'] == target)], over_time_metrics, timebin_metrics, plot_type='box',
                                                    pvals=pvals_df[(pvals_df['normalized'] == 0) & (pvals_df['outcome'] == target)])
fig1.suptitle(f'{target}: Pupillometry metrics over timebins (not normalized)', fontsize=16, y=0.9)

In [ ]:
# fig1.savefig(os.path.join(output_dir, f'{target}_pupillometry_metrics_over_timebins_not_normalized.tiff'), format='tiff', dpi=300)

In [ ]:
target = 'DCI_ischemia'
fig2, axes = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 1)
                                                    & (pupillometry_df['outcome'] == target)], over_time_metrics, timebin_metrics, plot_type='box',
                                                    pvals=pvals_df[(pvals_df['normalized'] == 1) & (pvals_df['outcome'] == target)])
fig2.suptitle(f'{target}: Pupillometry metrics over timebins (normalized)', fontsize=16, y=0.9)

In [ ]:
# fig2.savefig(os.path.join(output_dir, f'{target}_pupillometry_metrics_over_timebins_normalized.tiff'), format='tiff', dpi=300)

DCI infarct

In [ ]:
target = 'DCI_infarct'
fig3, axes = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 0) 
                                                    & (pupillometry_df['outcome'] == target)], over_time_metrics, timebin_metrics, plot_type='box',
                                                    pvals=pvals_df[(pvals_df['normalized'] == 0) & (pvals_df['outcome'] == target)])
fig3.suptitle(f'{target}: Pupillometry metrics over timebins (not normalized)', fontsize=16, y=0.9)

In [ ]:
# fig3.savefig(os.path.join(output_dir, f'{target}_pupillometry_metrics_over_timebins_not_normalized.tiff'), format='tiff', dpi=300)

In [ ]:
target = 'DCI_infarct'
fig4, axes = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 1)
                                                    & (pupillometry_df['outcome'] == target)], over_time_metrics, timebin_metrics, plot_type='box',
                                                    pvals=pvals_df[(pvals_df['normalized'] == 1) & (pvals_df['outcome'] == target)])
fig4.suptitle(f'{target}: Pupillometry metrics over timebins (normalized)', fontsize=16, y=0.9)

In [ ]:
# fig4.savefig(os.path.join(output_dir, f'{target}_pupillometry_metrics_over_timebins_normalized.tiff'), format='tiff', dpi=300)

Decompose figure into NPI and CV

In [ ]:
# for target in ['DCI_ischemia', 'DCI_infarct']:
#     for metric in ['NPI', 'CV']:
#         selected_timebin_metrics = [m for m in timebin_metrics if metric in m]
#         fig, axes = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 0) 
#                                                         & (pupillometry_df['outcome'] == target)], over_time_metrics, selected_timebin_metrics, plot_type='box',
#                                                         pvals=pvals_df[(pvals_df['normalized'] == 0) & (pvals_df['outcome'] == target)])
#         fig.suptitle(f'{target}: Pupillometry {metric} over timebins (not normalized)', fontsize=16, y=0.9)
#         # adjust figsize 20, 40
#         fig.set_size_inches(20, 40)
#         fig.savefig(os.path.join(output_dir, f'{target}_{metric}_pupillometry_metrics_over_timebins_not_normalized.tiff'), format='tiff', dpi=300)
#         
#         fig, axes = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 1)
#                                                         & (pupillometry_df['outcome'] == target)], over_time_metrics, selected_timebin_metrics, plot_type='box',
#                                                         pvals=pvals_df[(pvals_df['normalized'] == 1) & (pvals_df['outcome'] == target)])
#         fig.suptitle(f'{target}: Pupillometry {metric} over timebins (normalized)', fontsize=16, y=0.9)
#         fig.set_size_inches(20, 40)
#         fig.savefig(os.path.join(output_dir, f'{target}_{metric}_pupillometry_metrics_over_timebins_normalized.tiff'), format='tiff', dpi=300)


In [ ]:
sns.set_theme(style="whitegrid", context="paper", font_scale = 1)
cm = 1/2.54  # centimeters in inches
main_fig = plt.figure(figsize=(26 * cm, 20 * cm))

tick_label_size = 6
label_font_size = 7
subplot_number_font_size = 9
suptitle_font_size = 10
plot_subplot_titles = True
wspace = 0.15

target = 'DCI_ischemia'
selected_timebin_metrics = ['CV_inter_eye_min_timebin_max', 'CV_inter_eye_min_timebin_min', 'NPI_inter_eye_min_timebin_max']

subfigs = main_fig.subfigures(2, 1, height_ratios=[1, 1])

# Not normalized
fig1, axes1 = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 0) 
                                                & (pupillometry_df['outcome'] == target)], ['min', 'max', 'max'], selected_timebin_metrics, plot_type='box',
                                                pvals=pvals_df[(pvals_df['normalized'] == 0) & (pvals_df['outcome'] == target)], alpha=0.5,
                                                plot_legend=True, tick_label_size=tick_label_size, label_font_size=label_font_size, fig=subfigs[1])
# set subplot titles
axes1[0, 0].set_title('D. CV (inter-eye min, max in timebin)', horizontalalignment='left', x=-0.1, fontsize=subplot_number_font_size)
axes1[0, 1].set_title('E. CV (inter-eye min, min in timebin)', horizontalalignment='left', x=-0.1, fontsize=subplot_number_font_size)
axes1[0, 2].set_title('F. NPI (inter-eye min, max in timebin)', horizontalalignment='left', x=-0.1, fontsize=subplot_number_font_size)
subfigs[1].suptitle(f'II. Not normalized', fontsize=suptitle_font_size, horizontalalignment='left', x=0.1, y=1.)
subfigs[1].subplots_adjust(wspace=wspace)

# set ylims of CV to 0, 5.
for ax in axes1.flatten()[0:2]:
    ax.set_ylim(0, 4.75)

# Normalized
fig2, axes2 = plot_metric_distributions_over_timebins(pupillometry_df[(pupillometry_df['normalized'] == 1)
                                                & (pupillometry_df['outcome'] == target)], ['min', 'max', 'max'], selected_timebin_metrics, plot_type='box',
                                                pvals=pvals_df[(pvals_df['normalized'] == 1) & (pvals_df['outcome'] == target)], alpha=0.7,
                                                plot_legend=True, tick_label_size=tick_label_size, label_font_size=label_font_size, fig=subfigs[0])
# set subplot titles
axes2[0, 0].set_title('A. CV (inter-eye min, max in timebin)', horizontalalignment='left', x=-0.1, fontsize=subplot_number_font_size)
axes2[0, 1].set_title('B. CV (inter-eye min, min in timebin)', horizontalalignment='left', x=-0.1, fontsize=subplot_number_font_size)
axes2[0, 2].set_title('C. NPI (inter-eye min, max in timebin)', horizontalalignment='left', x=-0.1, fontsize=subplot_number_font_size)
subfigs[0].suptitle(f'I. Normalized', fontsize=suptitle_font_size, horizontalalignment='left', x=0.1, y=1.)
subfigs[0].subplots_adjust(wspace=wspace)

# set ylims to 0, 1.6
for ax in axes2.flatten():
    ax.set_ylim(0, 1.6)

main_fig.show()

In [ ]:
# main_fig.savefig(os.path.join(output_dir, f'main_metrics_boxplot_rescaled.tiff'), format='tiff', dpi=600, bbox_inches='tight')